In [1]:
import jieba,os,re,json
from gensim import corpora, models, similarities

# LDA model

In [3]:
label_dict = {} # record [label ID, occurence]

def process_label(data, msg):
    label_raw = [i[1] for i in data]
    label = [i for n, i in enumerate(label_raw) if i not in label_raw[:n]]
    if msg is 'train':
      for l in label_raw:
        if l not in label_dict:
          label_dict[l] = 1
        else:
          label_dict[l] += 1
    print('number of ' + msg + ' topics: ' + str(len(label)))
    print('number of ' + msg + ' instance: ' + str(len(data)))


"""Preprocess data_full.json"""
train_data = []
test_data = []
with open("./data_full.json") as f:
  data = json.load(f)
  train_data = data["train"]
  train_data = train_data[:1200]
  test_data = train_data[1::2]
  train_data = train_data[::2]
  process_label(train_data, 'train')

with open("./data_text.txt", "w") as f2:
  f2.truncate(0)
  for item in train_data:
    str_item = item[1] + '\t\t\t' + item[0] + '\n'
    f2.write(str_item)

with open("./test.txt", "w") as f2:
  f2.truncate(0)
  for item in test_data:
    str_item = item[1] + '\t\t\t' + item[0] + '\n'
    f2.write(str_item)
     

print(label_dict)

number of train topics: 12
number of train instance: 600
{'translate': 50, 'transfer': 50, 'timer': 50, 'definition': 50, 'meaning_of_life': 50, 'insurance_change': 50, 'find_phone': 50, 'travel_alert': 50, 'pto_request': 50, 'improve_credit_score': 50, 'fun_fact': 50, 'change_language': 50}


In [4]:
"""Create stopwords list"""
def stopwordslist():
    stopwords = [line.strip() for line in open('./stopwords_eng.txt',encoding='UTF-8').readlines()]
    return stopwords

"""Remove stopwords"""
def seg_depart(sentence):
    sentence_depart = jieba.cut(sentence.strip())
    stopwords = stopwordslist()
    outstr = ''
    for word in sentence_depart:
        if word not in stopwords:
            outstr += word
            outstr += " "      
    return outstr

if os.path.exists('./data_text.txt'):
    filename = "./data_text.txt"
    outfilename = "./result.txt"
    inputs = open(filename, 'r', encoding='UTF-8')
    outputs = open(outfilename, 'w', encoding='UTF-8')

    # Remove non-ASCII chars
    for line in inputs:
        line = line.split('\t\t\t')[1]
        line = re.sub(r'[^\x00-\x7F]','', line)
        line_seg = seg_depart(line.strip())
        outputs.write(line_seg.strip() + '\n')
    
    outputs.close()
    inputs.close()
    print("Process stopwords successful！！！")

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.008 seconds.
Prefix dict has been built successfully.


Process stopwords successful！！！


In [16]:
"""Convert output into the style needed by gensim"""
fr = open('./data_text.txt', 'r',encoding='utf-8')
train = []
for line in fr.readlines():
    line = line.split('\t\t\t')[1]
    line = re.sub(r'[^\x00-\x7F]','', line)
    line = seg_depart(line.strip())
    line = [word.strip() for word in line.split(' ')]
    train.append(line)
    
"""Build word frequency matrix and train the LDA model"""
dictionary = corpora.Dictionary(train)
# corpus[0]: [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1),...]
# corpus turn every conversation into ID，each element shows a word in the corresponding conversation,
# together with its frequency
corpus = [dictionary.doc2bow(text) for text in train]
print(corpus)

lda = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=12)
topic_list = lda.print_topics(12)

[[(0, 29), (1, 1), (2, 1), (3, 1)], [(0, 19), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1)], [(0, 25), (5, 1), (9, 1), (10, 1), (11, 1)], [(0, 27), (4, 2), (5, 1), (12, 1), (13, 1), (14, 1), (15, 1)], [(0, 25), (4, 2), (5, 1), (7, 1), (16, 1), (17, 1), (18, 1)], [(0, 21), (4, 3), (5, 1), (19, 1), (20, 1), (21, 1)], [(0, 29), (9, 1), (18, 1), (22, 1), (23, 1), (24, 1)], [(0, 29), (7, 1), (18, 1), (25, 1), (26, 1), (27, 1)], [(0, 17), (28, 1), (29, 1)], [(0, 13), (7, 1), (30, 1)], [(0, 7), (7, 1), (15, 1), (31, 1)], [(0, 19), (28, 1), (29, 1), (32, 1)], [(0, 13), (7, 1), (31, 1)], [(0, 15), (21, 1)], [(0, 7), (15, 1), (28, 1), (31, 1)], [(0, 13), (3, 1), (7, 1)], [(0, 13), (7, 1), (15, 1), (17, 1), (33, 1)], [(0, 13), (18, 1), (34, 1), (35, 1)], [(0, 15), (22, 1), (27, 1)], [(0, 13), (2, 1), (27, 1)], [(0, 9), (7, 1), (15, 1), (28, 1), (31, 1)], [(0, 11), (36, 1), (37, 1), (38, 1), (39, 1)], [(0, 7), (7, 1), (15, 1), (31, 1)], [(0, 13), (21, 1), (31, 1), (40, 1)], [(0, 13), (4, 1), (36, 1), (3

In [8]:
"""Retrive the topic of the test data"""  
file_test = "./test.txt"
msg_test = open(file_test, 'r', encoding='UTF-8')
    
test = []
labels = []
# Convert to correct format       
for line in msg_test:
    line_arr = line.split('\t\t\t')
    labels.append(line_arr[0])
    line = re.sub(r'[^\x00-\x7F]','', line_arr[1])
    line_seg = seg_depart(line.strip())
    line_seg = [word.strip() for word in line_seg.split(' ')]
    test.append(line_seg)    
    
# Turn converstation into ID    
corpus_test = [dictionary.doc2bow(text) for text in test]

# Get topic distribution of the test conversation
topics_test = lda.get_document_topics(corpus_test) 
correct = 0 
for i in range(len(labels)):
    label_ID = label_dict[labels[i]]
    for item in topics_test[i]:
      if label_ID is item[0]:
        correct += 1

    # print('The label ID is: '+ str(label_ID) + '\n')
    # print(topics_test[i],'\n')

print('Accuracy is : ' + str(correct / (len(labels))))

fr.close()
msg_test.close()

Accuracy is : 0.4066666666666667


# LDA with significance score

In [9]:
with open("./data_bias.json") as f:
  train_data = []
  test_data = []
  data = json.load(f)
  with open("./data_bias_text.txt", "w") as f2:
    data_list = ["insurance", "timer","transfer", "translate", "definition"]
    for i in range(len(data_list)):
      topic = data[data_list[i]]
      for j in topic:    
        str_item = data_list[i] +  '\t\t\t' + j[0]  + j[0]  + '\t\t\t' + j[1] + '\n'
        f2.write(str_item)
with open("./data_bias_text_no_significance.txt", "w") as f3:
  data_list = ["insurance", "timer","transfer", "translate", "definition"]
  for i in range(len(data_list)):
    topic = data[data_list[i]]
    for j in topic:    
      str_item = data_list[i] +  '\t\t\t' + j[0]  + '\t\t\t' + j[1] + '\n'
      f3.write(str_item)

In [10]:
"""Convert output into the style needed by gensim"""
fr = open('./data_bias_text.txt', 'r',encoding='utf-8')
train_bias = []
for line in fr.readlines():
    line = line.split('\t\t\t')[1]
    line = re.sub(r'[^\x00-\x7F]','', line)
    line = seg_depart(line.strip())
    line = [word.strip() for word in line.split(' ')]
    train_bias.append(line)
train_bias_slice = train_bias[::2]
    
"""Build word frequency matrix and train the LDA model"""
dictionary = corpora.Dictionary(train_bias_slice)
# corpus turn every conversation into ID，each element shows a word in the corresponding conversation,
# together with its frequency
corpus = [dictionary.doc2bow(text) for text in train_bias_slice]

lda = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=12)
topic_list = lda.print_topics(5)
print("The distribution of the 5 topics are as following：\n")
for topic in topic_list:
    print(topic)

The distribution of the 5 topics are as following：

(6, '0.023*"" + 0.023*"insurance" + 0.023*""" + 0.023*"milky" + 0.023*"money" + 0.023*"timer" + 0.023*"transfer" + 0.023*"six" + 0.023*"definition" + 0.023*"account"')
(5, '0.797*"" + 0.015*"insurance" + 0.012*""" + 0.008*"timer" + 0.008*"transfer" + 0.006*"rental" + 0.006*"5" + 0.006*"dollars" + 0.006*"obtain" + 0.006*"checking"')
(4, '0.023*"" + 0.023*"milky" + 0.023*"insurance" + 0.023*"money" + 0.023*"transfer" + 0.023*"timer" + 0.023*"set" + 0.023*""" + 0.023*"concept" + 0.023*"minute"')
(0, '0.023*"" + 0.023*"insurance" + 0.023*"milky" + 0.023*"money" + 0.023*"timer" + 0.023*"six" + 0.023*"define" + 0.023*"transfer" + 0.023*"definition" + 0.023*"concept"')
(8, '0.023*"" + 0.023*"milky" + 0.023*"insurance" + 0.023*"transfer" + 0.023*"money" + 0.023*"six" + 0.023*"timer" + 0.023*"minute" + 0.023*"definition" + 0.023*"account"')


In [11]:
"""Convert output into the style needed by gensim"""
fr = open('./data_bias_text_no_significance.txt', 'r',encoding='utf-8')
train_no_sig = []
for line in fr.readlines():
    line = line.split('\t\t\t')[1]
    line = re.sub(r'[^\x00-\x7F]','', line)
    line = seg_depart(line.strip())
    line = [word.strip() for word in line.split(' ')]
    train_no_sig.append(line)
train_no_sig = train_no_sig[::2]
    
"""Build word frequency matrix and train the LDA model"""
dictionary = corpora.Dictionary(train_no_sig)
# corpus turn every conversation into ID，each element shows a word in the corresponding conversation,
# together with its frequency
corpus = [dictionary.doc2bow(text) for text in train_no_sig]

lda = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=12)
topic_list = lda.print_topics(5)
print("The distribution of the 5 topics are as following：\n")
for topic in topic_list:
    print(topic)

The distribution of the 5 topics are as following：

(6, '0.032*"" + 0.032*"transfer" + 0.032*"insurance" + 0.032*"define" + 0.032*"money" + 0.032*"account" + 0.032*"set" + 0.032*"timer" + 0.032*"math" + 0.032*"milky"')
(9, '0.032*"" + 0.032*"define" + 0.032*"transfer" + 0.032*"insurance" + 0.032*"account" + 0.032*"set" + 0.032*"milky" + 0.032*"money" + 0.032*"timer" + 0.032*"definition"')
(10, '0.032*"" + 0.032*"transfer" + 0.032*"insurance" + 0.032*"account" + 0.032*"define" + 0.032*"set" + 0.032*"money" + 0.032*"milky" + 0.032*"timer" + 0.032*"definition"')
(2, '0.032*"" + 0.032*"define" + 0.032*"insurance" + 0.032*"transfer" + 0.032*"money" + 0.032*"milky" + 0.032*"set" + 0.032*"account" + 0.032*"timer" + 0.032*"definition"')
(5, '0.032*"" + 0.032*"transfer" + 0.032*"insurance" + 0.032*"define" + 0.032*"set" + 0.032*"account" + 0.032*"timer" + 0.032*"milky" + 0.032*"math" + 0.032*"money"')


In [13]:
# Turn converstation into ID
def calculate_acc(test, bias):
  corpus_test = [dictionary.doc2bow(text) for text in test]

  # Get topic distribution of the test conversation
  topics_test = lda.get_document_topics(corpus_test) 
  for i in range(len(topics_test)):
    correct = 0 
    for i in range(5 * 2): # 2 test cases for each category
        for item in topics_test[i]:
          if label_ID is item[0]:
            correct += 1
  print('Accuracy is : ' + str(correct / (len(labels))))

calculate_acc(train_bias[1::2], True)
calculate_acc(train_bias[1::2], False)

Accuracy is : 0.2
Accuracy is : 0.3


# Biterm model

In [17]:
!pip install bitermplus
!pip install tmplot
import bitermplus as btm
import numpy as np
import pandas as pd
import tmplot as tmp

     |████████████████████████████████| 614 kB 5.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for bitermplus: filename=bitermplus-0.6.10-cp37-cp37m-linux_x86_64.whl size=931272 sha256=566200dae6b6b9c9b7c1c1175ffb1eac89774ca53224920bd0998cf937611a09
  Stored in directory: /root/.cache/pip/wheels/2b/1a/8d/58328971ebe27f346b2e0c05ea9fe45bc67986310557074141
Successfully built bitermplus
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 16.3 MB 5.5 MB/s 
  Created wheel for tmplot: filename=tmplot-0.0.5-py3-none-any.whl size=13967 sha256=48c5159a5365dff4a32ed06dd23258b9d44197871c29a8d77c41380d9eb29f59
  Stored in directory: /root/.cache/pip/wheels/cf/46/a6/f91b4c6750de454081fa0526b4a17850692cb1fe295011c787
Successfully built tmplot


In [19]:
with open("./data_full.json") as f:
    data = json.load(f)
    train_data = data["train"]
    train_data = train_data[:1200]
    train_data = train_data[::2]
    process_label(train_data, 'train')

    with open("./data_text.txt", "w") as f2:
      f2.truncate(0)
      for item in train_data:
        str_item = item[0] + '\n'
        f2.write(str_item)
    


number of train topics: 12
number of train instance: 600


In [20]:
# IMPORTING DATA
print(label_dict)
df = pd.read_csv(
    './data_text.txt', header=None, names=['texts'])
texts = df['texts'].str.strip().tolist()

# Preprocessing
# Obtaining terms frequency in a sparse matrix and corpus vocabulary
X, vocabulary, vocab_dict = btm.get_words_freqs(texts)
tf = np.array(X.sum(axis=0)).ravel()

# Vectorizing documents
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
docs_lens = list(map(len, docs_vec))

# Generating biterms
biterms = btm.get_biterms(docs_vec)

# INITIALIZING AND RUNNING MODEL
model = btm.BTM(
    X, vocabulary, seed=12321, T=12, M=20, alpha=50/8, beta=0.01)
model.fit(biterms, iterations=20)
p_zd = model.transform(docs_vec)

# METRICS
# perplexity = btm.perplexity(model.matrix_topics_words_, p_zd, X, 8)
# coherence = btm.coherence(model.matrix_topics_words_, X, M=20)
# or
perplexity = model.perplexity_
coherence = model.coherence_

tmp.report(model=model, docs=texts)

{'translate': 200, 'transfer': 200, 'timer': 200, 'definition': 200, 'meaning_of_life': 200, 'insurance_change': 200, 'find_phone': 100, 'travel_alert': 100, 'pto_request': 100, 'improve_credit_score': 100, 'fun_fact': 100, 'change_language': 100}


100%|██████████| 551/551 [00:00<00:00, 122609.24it/s]
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:827: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  FutureWarning,
